In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("tripdata_2017-01.csv")
data['tpep_pickup_datetime'] = data['tpep_pickup_datetime'].astype('datetime64[ns]') 
data['tpep_pickup_hour'] = data.tpep_pickup_datetime.dt.hour
data['tpep_pickup_hour'].unique()
col         = 'tpep_pickup_hour'
conditions  = [ (data[col] >= 0) & (data[col] < 6), 
                (data[col]>= 6) & (data[col] < 12), 
               (data[col]>= 12) & (data[col] < 18),
               (data[col]>= 18) & (data[col] <= 24) 
  ]
choices     = [ 'N', 'M', 'A','E']
    
data["tpep_pickup_hour_slots"] = np.select(conditions, choices, default=np.nan)
data['tpep_pickup_hour_slots'].unique()
print(data['tpep_pickup_hour_slots'].value_counts())

N    102841
E     98568
A     82910
M     37000
Name: tpep_pickup_hour_slots, dtype: int64


In [ ]:
!pip install pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import when, col

spark = SparkSession.builder.getOrCreate()
data_sp = spark.read.csv('tripdata_2017-01.csv', header = True)
#data_sp.printSchema()
data_sp = data_sp.withColumn("tpep_pickup_timestap",to_timestamp("tpep_pickup_datetime")) 
data_sp = data_sp.withColumn("hour", hour(to_timestamp("tpep_pickup_timestap","yyyy-MM-dd HH:mm:ss")))
data_sp = data_sp.withColumn("slots_hour", when((col("hour") >= 0) & (col("hour") < 6),"N")
                                 .when((col("hour") >= 6) & (col("hour") < 12),"M")
                                 .when((col("hour") >= 12) & (col("hour") < 18),"A")
                                 .when((col("hour") >= 18) & (col("hour") <= 24),"E")
                                 .when(col("hour").isNull() ,""))
data_sp.groupBy('slots_hour').count().show()

     |█████████████▊                  | 121.0 MB 1.4 MB/s eta 0:01:56